# テスト用ノートブック

### 参考資料
https://qiita.com/tatsuki-tsuchiyama/items/4c996ce740cc0c607d35

In [8]:
import configparser
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.sql_database import SQLDatabase

In [9]:
#from langchain import SQLDatabaseChain
#from langchain_community.utilities import SQLDatabaseChain
from langchain_experimental.sql import SQLDatabaseChain

In [10]:
# 設定ファイルの読み込み
config = configparser.ConfigParser() #Configのハンドル設定
config.read(".././private/config.ini")

#OpenAI部分の設定参照
openai_config = config["OPENAI"] 
input_openai_api_key = openai_config["OPENAI_API_KEY"]

In [11]:
#データベースとモデルの設定
db = SQLDatabase.from_uri("sqlite:///.././db/chinook.db")

In [12]:
llm = ChatOpenAI(
    openai_api_key=input_openai_api_key,
    temperature=0,
    model_name="gpt-3.5-turbo",
    streaming=True,
)

C:\Users\tshimada\AppData\Local\Temp\ipykernel_7100\3713238177.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [15]:
agent_executor = SQLDatabaseChain(
    llm=llm,
    database=db,
    verbose=True
)

c:\01_program\01_github\101_004_Webapp_SQLRAG\venv\llm_SQL_app\Lib\site-packages\langchain_experimental\sql\base.py:77: UserWarning: Directly instantiating an SQLDatabaseChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [16]:
agent_executor = SQLDatabaseChain.from_llm(
    llm,
    db,
    verbose=True
)

In [17]:
agent_executor.run("いくつのデータが存在しますか？")

C:\Users\tshimada\AppData\Local\Temp\ipykernel_7100\836053850.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent_executor.run("いくつのデータが存在しますか？")
c:\01_program\01_github\101_004_Webapp_SQLRAG\venv\llm_SQL_app\Lib\site-packages\langchain_community\utilities\sql_database.py:379: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  sample_rows_result = connection.execute(command)  # type: ignore




> Entering new SQLDatabaseChain chain...
いくつのデータが存在しますか？
SQLQuery:SELECT COUNT(*) AS TotalData FROM albums
SQLResult: [(347,)]
Answer:データは347件存在します。
> Finished chain.


'データは347件存在します。'

In [14]:
agent_executor.run("データ項目を教えて")



> Entering new SQLDatabaseChain chain...
データ項目を教えて
SQLQuery:SELECT name
FROM sqlite_master
WHERE type = 'table'
LIMIT 5;
SQLResult: [('albums',), ('sqlite_sequence',), ('artists',), ('customers',), ('employees',)]
Answer:albums, sqlite_sequence, artists, customers, employees
> Finished chain.


'albums, sqlite_sequence, artists, customers, employees'

In [16]:
agent_executor.run("playlistedのデータを見せて")



> Entering new SQLDatabaseChain chain...
playlistedのデータを見せて
SQLQuery:SELECT "PlaylistId", "TrackId"
FROM playlist_track
LIMIT 5;

SQLResult: [(1, 3402), (1, 3389), (1, 3390), (1, 3391), (1, 3392)]
Answer:Playlistedのデータは以下の通りです:
(1, 3402)
(1, 3389)
(1, 3390)
(1, 3391)
(1, 3392)
> Finished chain.


'Playlistedのデータは以下の通りです:\n(1, 3402)\n(1, 3389)\n(1, 3390)\n(1, 3391)\n(1, 3392)'